In [1]:
def read_restart(file_name_restart):
    with open(file_name_restart, 'r') as file:
        status = False
        count = 0
        complex_lst = []
        for line in file.readlines():
            if line == '#All Complexes and their components \n':
                status = True
            if status:
                if count % 8 == 7:
                    info = line.split()
                    temp_lst = []
                    for i in range(len(info)):
                        if i != 0:
                            temp_lst.append(int(info[i]))
                    complex_lst.append(temp_lst)        
                count += 1
            if line == '#Observables \n':
                break
    print('The total number of complexes is',len(complex_lst))
    return complex_lst

In [2]:
import pandas as pd

def pdb_to_df(file_name_pdb):
    df = pd.DataFrame(columns = ['Protein_Num','Protein_Name'])
    with open(file_name_pdb, 'r') as file:
        index = 0
        for line in file.readlines():
            line = line.split(' ')
            if line[0] == 'ATOM':
                info = []
                for i in line:
                    if i != '':
                        info.append(i)
                df.loc[index, 'Protein_Num'] = int(info[4])
                df.loc[index, 'Protein_Name'] = info[3]
            index += 1
        df = df.dropna()
        df = df.reset_index(drop = True)
    return df

In [3]:
def complex_df_gen(pdb_df, complex_lst):
    name_lst = list(pdb_df['Protein_Name'])
    name_lst_ = []
    for i in name_lst:
        if i not in name_lst_:
            name_lst_.append(i)
    column_lst = []
    for i in name_lst_:
        column_lst.append(i)
    column_lst.append('Protein_Num')
    complex_df = pd.DataFrame(columns = column_lst)
    index = 0
    for i in complex_lst:
        complex_df.loc[index] = 0
        complex_df.loc[index,'Protein_Num'] = str(i)
        for j in i:
            for indexs in pdb_df.index: 
                for k in range(len(pdb_df.loc[indexs].values)): 
                    if(pdb_df.loc[indexs].values[k] == j): 
                        col = pdb_df.loc[indexs,'Protein_Name']
                        complex_df.loc[index,col] += 1
                        break
                else:
                    continue
                break
        index += 1 
    return complex_df


In [4]:
def find_complex_df(complex_df, num_lst, pdb_df):
    protein_name = []
    for i in range(len(pdb_df)):
        if pdb_df.loc[i, 'Protein_Name'] not in protein_name:
            protein_name.append(pdb_df.loc[i, 'Protein_Name'])
    complex_df['Num_List'] = ''
    for i in range(complex_df.shape[0]):
        lst = []
        for j in range(complex_df.shape[1]-2):
            lst.append(complex_df.iloc[i, j])
        complex_df.loc[i, 'Num_List'] = str(lst)
    num_lst_str = str(num_lst)
    protein_remain = []
    for i in range(complex_df.shape[0]):
        if complex_df.loc[i, 'Num_List'] == num_lst_str:
            string = complex_df.loc[i, 'Protein_Num']
            string = string.strip('[').strip(']').split(',')
            for i in string:
                protein_remain.append(int(i))           
    return protein_remain



In [5]:
def new_pdb(file_name_pdb, protein_remain):
    with open(file_name_pdb, 'r') as file:
        write_lst = []
        for line in file.readlines():
            line_ = line.split(' ')
            if line_[0] == 'TITLE':
                write_lst.append(line)
            elif line_[0] == 'CRYST1':
                write_lst.append(line)
            elif line_[0] == 'ATOM':
                info = []
                for i in line_:
                    i.strip('\n')
                    if i != '':
                        info.append(i)
                info[9] = info[9].strip('\n')
                if int(info[4]) in protein_remain:
                    write_lst.append(line)
    with open('output_file.pdb','w') as file_:
        file_.seek(0)
        file_.truncate()
        for i in write_lst:
            file_.writelines(i)
    return 0


In [6]:
def locate_position_restart(file_name_pdb, num_lst, file_name_restart = 'restart.dat'):
    print('Reading restart.dat...')
    complex_lst = read_restart(file_name_restart)
    print('Reading files complete!')
    print('Reading PDB files...')
    pdb_df = pdb_to_df(file_name_pdb)
    print('Reading files complete!')
    print('Finding complexes...')
    complex_df = complex_df_gen(pdb_df, complex_lst)
    print('Finding complexes complete!')
    print('Writing new PDB files...')
    protein_remain = find_complex_df(complex_df, num_lst, pdb_df)
    new_pdb(file_name_pdb, protein_remain)
    print('PDB writing complete!(named as output_file.pdb)')

In [8]:
locate_position_restart(file_name_pdb = '9999999.pdb', num_lst = [18], file_name_restart = 'restart.dat')

Reading restart.dat...
The total number of complexes is 22
Reading files complete!
Reading PDB files...
Reading files complete!
Finding complexes...
Finding complexes complete!
Writing new PDB files...
PDB writing complete!(named as output_file.pdb)
